In [1]:
import numpy as np 
import pandas as pd 
from plotly import graph_objs as go

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/Womens Clothing E-Commerce Reviews.csv')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [4]:
df.shape

(23486, 11)

In [6]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [7]:
del df['Unnamed: 0']

In [ ]:
df.head()

In [9]:
df.isnull().sum()

Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [10]:
df.isnull().sum().sort_values(ascending=False)

Title                      3810
Review Text                 845
Division Name                14
Department Name              14
Class Name                   14
Clothing ID                   0
Age                           0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
dtype: int64

In [11]:
df = df.dropna(subset = ['Review Text', 'Division Name', 'Department Name', 'Class Name'])

In [12]:
df.isnull().sum().sort_values(ascending=False)

Title                      2966
Clothing ID                   0
Age                           0
Review Text                   0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                 0
Department Name               0
Class Name                    0
dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

import re
import string

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
data = df[['Review Text', 'Recommended IND']]

In [15]:
data.head()

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [16]:
# Cleaning the text
def clean_text(text):
    '''Make text lowercase, remove text in square brackets, 
    remove links, remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [17]:
data['Review Text'] = data['Review Text'].apply(lambda x:clean_text(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
data.head()

,Review Text,Recommended IND
0,absolutely wonderful silky and sexy and comfo...,1
1,love this dress its sooo pretty i happened t...,1
2,i had such high hopes for this dress and reall...,0
3,i love love love this jumpsuit its fun flirty ...,1
4,this shirt is very flattering to all due to th...,1


In [21]:
X = data.drop('Recommended IND', axis=1)
y = data['Recommended IND']

# Spliting train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify=y,
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    shuffle=True)

In [22]:
num_words = None

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train['Review Text'].tolist() + X_test['Review Text'].tolist()) 

In [23]:
word_index = tokenizer.word_index

In [24]:
# Encode training/test data sentences into sequences
X_train_seq = tokenizer.texts_to_sequences(X_train['Review Text'].tolist())
X_test_seq = tokenizer.texts_to_sequences(X_test['Review Text'].tolist())

In [25]:
# Get max training sequence length
max_len = max([len(x) for x in X_train_seq])

In [26]:
# Pad the training/test sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [27]:
# Output some results 
print("\nPadded training shape:", X_train_pad.shape)
print("\nPadded test shape:", X_test_pad.shape)


Padded training shape: (18102, 112)

Padded test shape: (4526, 112)


In [28]:
# Basic RNN 
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                    50,     # embeds it in a 50-dimensional vector
                    input_length=max_len))
model.add(SimpleRNN(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 112, 50)           1022500   
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               15100     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1,037,701
Trainable params: 1,037,701
Non-trainable params: 0
_________________________________________________________________


In [29]:
batch_size = 512

model.fit(X_train_pad, y_train, epochs=5, batch_size=batch_size)

Epoch 1/5
36/36 [==============================] - 11s 247ms/step - loss: 0.5337 - accuracy: 0.7553
Epoch 2/5
36/36 [==============================] - 9s 245ms/step - loss: 0.4565 - accuracy: 0.8187
Epoch 3/5
36/36 [==============================] - 9s 239ms/step - loss: 0.4123 - accuracy: 0.8214
Epoch 4/5
36/36 [==============================] - 12s 322ms/step - loss: 0.3007 - accuracy: 0.8702
Epoch 5/5
36/36 [==============================] - 13s 369ms/step - loss: 0.2311 - accuracy: 0.9033


In [31]:
scores = model.predict(X_test_pad)

142/142 [==============================] - 2s 12ms/step


In [32]:
# Setting up the evaluation metrics
def roc_auc(predictions,target):
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [33]:
print("AUC: %.2f%%" % (roc_auc(scores,y_test)))

AUC: 0.91%


In [34]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                    50,     # embeds it in a 50-dimensional vector
                    input_length=max_len))

model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 112, 50)           1022500   
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,083,001
Trainable params: 1,083,001
Non-trainable params: 0
_________________________________________________________________


In [35]:
batch_size = 512

model.fit(X_train_pad, y_train, epochs=5, batch_size=batch_size)

Epoch 1/5
36/36 [==============================] - 57s 1s/step - loss: 0.5129 - accuracy: 0.8095
Epoch 2/5
36/36 [==============================] - 48s 1s/step - loss: 0.3415 - accuracy: 0.8411
Epoch 3/5
36/36 [==============================] - 52s 1s/step - loss: 0.2400 - accuracy: 0.8969
Epoch 4/5
36/36 [==============================] - 48s 1s/step - loss: 0.1962 - accuracy: 0.9185
Epoch 5/5
36/36 [==============================] - 52s 1s/step - loss: 0.1686 - accuracy: 0.9316


In [36]:
scores = model.predict(X_test_pad)
print("AUC: %.2f%%" % (roc_auc(scores,y_test)))

142/142 [==============================] - 5s 33ms/step
AUC: 0.93%
